In [1]:
print(sys.executable)

C:\Users\Emanuel\AppData\Local\Programs\Python\Python38\python.exe


In [2]:
import wnbl
import pandas as pd
import matplotlib.pyplot as plt
import glob

current_comp_id = 28356

# need an initial URL so as to get list of all seasons and team IDs
url_comp = "https://hosted.wh.geniussports.com/embednf/WNBL/en/competition/{COMP}/statistics/team?&iurl=https%3A%2F%2Fwnbl.basketball%2Fstats%2F&_cc=1&_nv=1&_mf=1"

#
url_standings = "https://hosted.wh.geniussports.com/embednf/WNBL/en/competition/{COMP}/standings?&iurl=https%3A%2F%2Fwnbl.basketball%2Fstats%2F&_cc=1&_nv=1&_mf=1"

# team roster - selected by competition and team - table 1
url_roster = "https://wnbl.basketball/stats/?WHurl=%2Fcompetition%2F{COMP}%2Fteam%2F{TEAM}%2Froster"

# team statistics -selected by competition and team - table 3
# url_team = "https://wnbl.basketball/stats/?WHurl=%2Fcompetition%2F{COMP}%2Fteam%2F{TEAM}%2Fstatistics"
url_team = "https://hosted.wh.geniussports.com/embednf/WNBL/en/competition/{COMP}/team/{TEAM}/statistics?&iurl=https%3A%2F%2Fwnbl.basketball%2Fstats%2F&_cc=1&_nv=1&_mf=1"


In [3]:
# get seasons from the drop-down select elements

comp_id = str(current_comp_id)
urlc = url_comp.replace('{COMP}',comp_id)


In [ ]:
seasons = wnbl.get_seasons(url_comp)
seasons_df = pd.DataFrame(seasons,columns=['Season','Year','URL','CompID'])
seasons_df.to_csv('seasons.csv',index=False)

In [ ]:
# ## Need to loop through comps to teams lists - similar to players below

teams_df = pd.DataFrame()

for rec in seasons:
    # loop through each competition (season)

    comp_id = rec[3]
    urlc = url_comp.replace('{COMP}',comp_id)
    filename = 'teams_mapping_' + str(comp_id) + '.csv'

    dft = wnbl.get_teams(urlc)

    # save individual seasonal data file
    if not glob.glob(filename): dft.to_csv(filename,index=False)

    # append this season to master dataframe
    # dft['CompID'] = comp_id
    teams_df = teams_df.append(dft,index=False)

# keep unique rows only
teams_df = teams_df.drop_duplicates().reset_index(drop=True)


In [ ]:
standings_df = pd.DataFrame()

for rec in seasons:
    # loop through each competition (season)

    comp_id = rec[3]
    urlc = url_standings.replace('{COMP}',comp_id)
    filename = 'standings_' + str(comp_id) + '.csv'

    dfs = wnbl.get_standings(urlc)

    # save individual seasonal data file
    if not glob.glob(filename): dfs.to_csv(filename,index=False)

    # append this season to master dataframe
    dfs['CompID'] = comp_id
    standings_df = standings_df.append(dfs)

# remove logo column (NaN)
standings_df = standings_df.drop(standings_df.columns[1], axis=1)

In [ ]:
# standings don't use full team name so merge with teams data
standings_df = standings_df.merge(teams_df,on='TeamID',how='left')

In [ ]:
teams_df = standings_df[standings_df['CompID']==comp_id]
teams_df

In [ ]:

# for given competition, cycle through teams and get stats

players_df = pd.DataFrame()

for rec in seasons:

    comp_id = rec[3]   
    url1 = url_team.replace('{COMP}',comp_id)

    stats_df = pd.DataFrame()

    # process the teams in the selected season
    teams_df = standings_df[standings_df['CompID']==comp_id]

    for index, row in teams_df.iterrows():
        team_name = row['Team_y']
        team_id = row['TeamID']
        url2 = url1.replace('{TEAM}',str(team_id))

        dfp = wnbl.get_players(url2)

        dfp['TeamID'] = team_id
        stats_df = stats_df.append(dfp,ignore_index=True)

    filename = 'players_' + str(comp_id) + '.csv'
    if not glob.glob(filename): stats_df.to_csv(filename,index=False)

    stats_df['CompID'] = comp_id
    players_df = players_df.append(stats_df,ignore_index=True)
